# Description

Use incremental solving to compute the optimal solution and store agent-location-table in the original_plan.lp file

In [1]:
!python inc_orig.py encoding_orig.lp instances/env_17_multi_agent.lp --imax 100

inc-example version 1.0
Reading from encoding_orig.lp ...
Solving...
Solving...
Solving...
Solving...
Solving...
Solving...
Solving...
Solving...
Solving...
Solving...
Solving...
Solving...
Solving...
Solving...
Solving...
Solving...
Solving...
Solving...
Solving...
Solving...
Solving...
Solving...
Solving...
Solving...
Solving...
Solving...
Solving...
Solving...
Solving...
Solving...
Solving...
Solving...
Solving...
Solving...
Solving...
Solving...
Solving...
Answer: 1
orig(agent(0),(25,5),0,(25,5)) orig(agent(1),(25,6),0,(25,6)) orig(agent(1),(25,6),1,(25,6)) orig(agent(0),(25,5),1,(25,5)) orig(agent(0),(25,5),2,(25,4)) orig(agent(1),(25,6),2,(25,6)) orig(agent(1),(25,6),3,(25,5)) orig(agent(0),(25,4),3,(25,3)) orig(agent(0),(25,3),4,(25,2)) orig(agent(1),(25,5),4,(25,4)) orig(agent(1),(25,4),5,(25,3)) orig(agent(0),(25,2),5,(25,1)) orig(agent(0),(25,1),6,(24,1)) orig(agent(1),(25,3),6,(25,2)) orig(agent(1),(25,2),7,(25,1)) orig(agent(0),(24,1),7,(23,1)) orig(agent(0),(23,1),8,(22,1)

Compute conflict locations for agents (Locations which are passed by more than one agent)

In [2]:
!python conflict_locations.py

Conflict locations written to conflict_locations.lp


Do iterative solving with malfunctions

In [9]:
!python inc_delay.py encoding_delay.lp instances/env_17_multi_agent.lp --imax 100 --delay_rate 0.2 --min_duration 2 --max_duration 5

inc-example version 1.0
Reading from encoding_delay.lp ...
encoding_delay.lp:120:1-17: info: no atoms over signature occur in program:
  delayed/3

Solving...
Traceback (most recent call last):
  File "/Users/ekaterinavolosinova/opt/anaconda3/envs/potassco2/lib/python3.11/site-packages/clingo/application.py", line 359, in _pyclingo_application_main
    app.main(Control(control), [_to_str(files[i]) for i in range(size)])
  File "/Users/ekaterinavolosinova/ASP/train-scheduling/inc_delay.py", line 211, in main
    ctl.ground(parts)
  File "/Users/ekaterinavolosinova/opt/anaconda3/envs/potassco2/lib/python3.11/site-packages/clingo/control.py", line 481, in ground
    c_part.params[i] = sym._rep
                       ^^^^^^^^
AttributeError: 'int' object has no attribute '_rep'
*** ERROR: (inc-example): no message
UNKNOWN

Models       : 0+
Calls        : 2
Time         : 0.037s (Solving: 0.00s 1st Model: 0.00s Unsat: 0.00s)
CPU Time     : 0.234s
